In [1]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem

In [ ]:
import kafka.client_async

def _poll_patched(self, timeout):
    timeout = int(timeout) if timeout is not None else None
    ready = self._selector.select(timeout)
    return ready

# Monkey-patch the KafkaClient
kafka.client_async.KafkaClient._poll = _poll_patched

In [ ]:
consumer = KafkaConsumer(
    'demo_test',
     bootstrap_servers=['18.119.101.110:9092'], 
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
s3 = S3FileSystem()

In [ ]:
S3_BUCKET = 'kafka-stock-ak'
S3_PREFIX = 'alpha-stock/'

In [ ]:
def write_to_s3(data, timestamp):
    # Convert data to DataFrame
    df = pd.DataFrame([data])
    
    # Define S3 path
    s3_path = f"{S3_BUCKET}/{S3_PREFIX}{timestamp}.parquet"
    
    # Write DataFrame to S3 as Parquet
    with s3.open(s3_path, 'wb') as f:
        df.to_parquet(f)
    print(f"Data written to S3: {s3_path}")

In [ ]:
for message in consumer:
    try:
        # Parse the message value (JSON)
        data = json.loads(message.value.decode('utf-8'))
        
        # Extract timestamp from the data
        timestamp = data.get('timestamp', 'default')
        
        # Write data to S3
        write_to_s3(data, timestamp)
    
    except Exception as e:
        print(f"Error processing message: {e}")